# Plot and explore 2D case

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def e_return(weights: np.array, returns: np.array):
    return np.dot(weights, returns)

def _xtax(x: np.array, A: np.array):
    return np.matmul(np.transpose(np.matmul(x, A)), x)

def risk(weights: np.array, covariances: np.array):
    return np.sqrt(_xtax(weights, covariances))

mu = np.array([0.1, 0.2])
cov = np.array([[.15, -.05], [-.05, .3]])

weights = np.array([
    [x, 1.-x]
    for x in np.arange(0., 1., 0.01, float)
])

returns = np.matmul(weights, mu)
risks = [risk(w, cov) for w in weights]
sharpe = returns / risks

plt.scatter(risks, returns, c=sharpe)

plt.xlabel('risk')
plt.ylabel('return')

plt.xlim(0., 1.)
plt.ylim(0., .5)

plt.show()


# Get close data

In [ ]:

from ise_data_utils import to_dataframe_cache
from persistence import DependencySpec

ds = DependencySpec(["close"], period="daily")
df = to_dataframe_cache(ds).unstack(level=-1)

df.columns = df.columns.droplevel()
print(df)


In [ ]:
df.drop(columns=[col for col in df.columns if any(set(map(str, range(10))) & set(col))], inplace=True)

In [ ]:
import heapq
import pandas as pd
import random

def select_from_df(df: pd.DataFrame, n):
        
    na_rates = df.isna().sum() / df.shape[0]
    name_normal = lambda name: not any(set(range(10)) & set(name[-1]))

    best_many = heapq.nsmallest(min(10*n, len(df.columns)), na_rates.index, key=lambda ticker: na_rates[ticker] if name_normal(ticker) else 2.)
    best_few = random.choices(best_many, k=n)

    _selection = df[best_few]
    return _selection

def logify(df: pd.DataFrame) -> pd.DataFrame:
    """Apply ffill(5), dropna, diff and again dropna on df"""
    _logification = df.ffill(limit=5).bfill(limit=5).dropna()
    _logification = _logification.apply(np.log).diff().dropna()
    return _logification

def prep_select(df: pd.DataFrame, n: int):
    return logify(select_from_df(df, n))

df_selection = prep_select(df, 100)
print(df_selection)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def e_return(weights: np.array, returns: np.array):
    return np.exp(np.matmul(weights, returns) * 365) - 1.

def _xtax(x: np.array, A: np.array):
    return np.matmul(np.transpose(np.matmul(x, A)), x)

def risk(weights: np.array, covariances: np.array):
    result = np.sqrt(_xtax(weights, covariances))
    return result

def random_portfolio(expected_returns: np.array, covariance: np.array) -> np.array:
    return np.random.dirichlet(np.ones(cov.shape[0]))

from efficientfrontier import min_risk, max_sharpe

mu = df_selection.mean().values
cov = df_selection.cov().values

n_portfolios = 10000

weights = np.array([random_portfolio(mu, cov) for _ in range(n_portfolios)])

returns = e_return(weights, mu)
risks = np.array([risk(w, cov) for w in weights])
sharpe = returns / risks

plt.scatter(risks, returns, c=sharpe, s=.4)

least_risk = min_risk(mu, cov)
least_risk_e_ret = e_return(least_risk, mu)
least_risk_e_risk = risk(least_risk, cov)

plt.scatter([least_risk_e_risk], [least_risk_e_ret], marker='*')

highest_sharpe = max_sharpe(mu, cov)
highest_sharpe_e_ret = e_return(highest_sharpe, mu)
highest_sharpe_e_risk = risk(highest_sharpe, cov)

plt.scatter([highest_sharpe_e_risk], [highest_sharpe_e_ret], marker='*')

plt.xlabel('risk')
plt.ylabel('return')

plt.show()

In [ ]:
class Portfolio:
    weighting: pd.Series
    cash: float
    def __init__(self) -> None:
        self.k = 1.
        self.weighting = pd.Series(data=[], index=[])

    def validate(self) -> bool:
        return self.k > 1e-8

    def update(self, new_weighting: pd.Series, market: pd.Series):
        # Convert all we have to cash
        if self.weighting.sum() > .5:
            self.k *= market.multiply(self.weighting, fill_value=0.).sum()
        self.weighting = pd.Series(data=new_weighting, index=market.index)
        self.k /= market.multiply(self.weighting, fill_value=0.).sum()


In [ ]:
memory = 200
history_ix = df.index
least_risk = Portfolio()
most_sharpe = Portfolio()
rand_portf = Portfolio()
market_selection = prep_select(df, 100)
for i0 in range(len(market_selection.index) - memory):
    if i0 % 10 != 0:
        continue
    i1 = i0+memory
    ix = market_selection.index[i0:i1]
    df_selection = market_selection.loc[ix, :]

    mu = df_selection.mean().values
    cov = df_selection.cov().values

    today = df_selection.index[-1]
    market_today = df_selection.loc[today, :].fillna(0.).sparse.to_dense()
    market_today[market_today < 0.] = 0.

    if len(market_today) < market_selection.shape[1] / 2 or any(market_today.isna()):
        print("NA detected in market, skipping ahead")
        continue

    w_min_risk = min_risk(mu, cov)
    assert min(w_min_risk) >= 0.0, w_min_risk
    least_risk.update(w_min_risk, market_today)
    w_max_sharpe = max_sharpe(mu, cov)
    assert min(w_max_sharpe) >= 0.0, w_max_sharpe
    most_sharpe.update(w_max_sharpe, market_today)
    w_rand = random_portfolio(mu, cov)
    assert min(w_rand) >= 0.0, w_rand
    rand_portf.update(w_rand, market_today)

    print(f"{least_risk.k=:3.3f}, {most_sharpe.k=:3.3f}, {rand_portf.k=:3.3f}")